<a href="https://colab.research.google.com/github/AbhinavRobinson/sentdex_tut_dl/blob/master/R_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Import main libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNLSTM

# Loading Dataset
mnist = tf.keras.datasets.mnist

# Slicing the dataset into train and test
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalizing Data
x_train = x_train / 255.0
x_test = x_test / 255.0

# lets get the shape of the dataset
print(x_train.shape)
print(x_train[0].shape)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(28, 28)


In [0]:
# Just to make sure Google Colab is using K80 and not CPU (CUDNNLSTM dependency)
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16734125627796413949
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17127930880834976566
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 6400054780561811164
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11330115994
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17589851025127099591
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [0]:
model = Sequential()

"""
CUDNNLSTM speeds up train time from 5min per epoch 
to 2mins for entire training! :)
Make sure Runtime>Change Runtime>Hardware Acceleration>GPU
is enabled :D

NOTE:
CUDNNLSTM uses its own activation function "tanh"

Dont train this on your Laptop ;)
"""

# Adding a CuDNNLSTM Layer
model.add(CuDNNLSTM(128, input_shape=(x_train.shape[1:]), 
                    return_sequences=True))
# Adding a Dropout, "cos dats wat ya do"
model.add(Dropout(0.2))

# Adding the Second CuDNNLSTM Layer
model.add(CuDNNLSTM(128))
model.add(Dropout(0.2))

# Adding Dense Layer
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

# Adding Last Dense Layer, 10 since we have 10 classes
model.add(Dense(10, activation='softmax'))
# Model Structure Complete

# Lets compile
# Lets add a customized Adam optimizer, we add Decay to shrink learning rate
opt = tf.keras.optimizers.Adam(lr=1e-3, decay=1e-5)
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# Lets train the model
model.fit(x_train, y_train, epochs=3, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 40s 668us/sample - loss: 0.4180 - acc: 0.8727 - val_loss: 0.1709 - val_acc: 0.9481
Epoch 2/3
60000/60000 [==============================] - 40s 666us/sample - loss: 0.1257 - acc: 0.9668 - val_loss: 0.0887 - val_acc: 0.9748
Epoch 3/3
60000/60000 [==============================] - 40s 661us/sample - loss: 0.0836 - acc: 0.9773 - val_loss: 0.0630 - val_acc: 0.9819
